Section 1. Dirichlet Processes

In general a topic model can be described as a clustering procedure. The goal is to group related elements (documents) of some universal set (corpus). Before discussing how Dirichlet processes can be used to build topic models we will first discuss Dirichlet processes in general and how they can be used to do mixture of Gaussian clustering. In all parametric clustering procedures, such as k-means, GMM and LDA the number of clusters must be pre-specified.  One of the major advantages when using Dirichlet processes and in general nonparametric techniques is that the number of cluster can be learned from the algorithm. 
    
There are 3 common ways (all equivalent) to introduce Dirichlet processes: the Chinese Restaurant Process, the Polya Urn Model and the Stick Breaking Process. 

Section 1.1: The Chinese Restaurant Process

The Chinese Restaurant Process (CP) supposes that you have some number of customers (elements of a dataset) and one by one they each walk into a restaurant and sit at a table according the following rules:
    
   1. The first person sits at the first table.
   2. The second person sits at a new table with probability $\alpha / (1 + \alpha)$ and sits at the first table with probability $1 / (1 + \alpha)$.
   3. The n-th person sits at a new table with probability $\alpha / (n-1 + \alpha)$ and sits at the table k with probability $n_k / (n + \alpha)$ where $n_k$ is the number of people sitting at table k.

Note that the number of tables (clusters) is not fixed but instead grows as more customers (elements in the dataset) enter the restaurant. Also note that the probability of a new table (cluster) depend on a dispersion parameter $\alpha$. Also note that tables with a greater number of customers currently sitting at it the more likely new cusomers will sit at the table. Figure 1. Shows two simulations of the Chinese Restaurant Process with one thousand customers and two different values for the dispersion parameter. Note that the simulation with the greater dispersion parameter has more active tables (clusters).
    
 <img src="images/cp.jpg">
    
 Figure 1: This Figure shows two simulations of the chinese restaurant process. The top image is run with $\alpha=1.$ and the the bottom figure with $\alpha=100.$. We can clearly see that as the dispersion parameter increases the number of clusters is likely to increase.

Section 1.2: The Polya Urn Model

The Polya Urn model is very similiar to the Chinese Restraunt Process. It supposes that you start with an urn containing balls of color (clusters) x (note in this case colors are real numbers). You are given a base distribution F where F(x) is the probability of sampling x from F. The urn contrains $\alpha*F(x)$ balls of the specific color x. At each time step you then sample a ball from the urn and replace the ball along with a ball of identical color back in the urn. Note how drawing a ball of a certain color increases the probabiliy of a drawing a ball of that color similiar to how tables with more cusomters have greater probability of attracting new customers in CPs. Figure 2. shows three simulations of the Polya Urn model where the base distribution is Gaussian with mean 0 and unit variance. Note how as the dispersion parameter increases the result of the simulation looks more like the base distribution.
   <img src="images/polya.jpg">
   Figure 2: Shows the three simulations of the Polya Urn Process with 1000 balls and alpha being 1, 10, or 100. The base distriubtion is normal with mean 0 and unit variance. Note how as the dispersion parameter increaes the desnity more closely resembles the base distribution

Section 1.3: The Stickbreaking Process

A third and final way to simulate Dirichlet Processes is the stick breaking process. Imagine running the Chinese Restaurant Proccess to infinity recording the proportions of people at each table. Denote the proportion of table i as $p_i$. Instead of determing the proprotions by simulating the Chinese Restaurant Process (or Polya Urn) the stick breaking process is able to determine them directly through the following rules: 

1. We start with a stick of unit length.
2. We generate a random variable $\beta_1 ~ Beta(1, \alpha)$ and then break the stick at $\beta_1$ recording the length of the left end and saving the right end for the next step. The length of the left end of the stick is $p_1$ which is $\beta_1$
3. We generate another random variable $\beta_2 ~ Beta(1, \alpha)$ and break the stick again at $\beta_2$ again recording the length of the left end and saving the right end for the next step. The length of the left end is $p_2$ or $(1 - \beta_1) * \beta_2$
4. Continue

Note that assigning customers to table k using a CP is equivalent to assining a customer to table k with probability $w_k$. Figure 3. shows the results of two simulations of the Stick Breaking process for two different values of the dispersion parameter.

<img src="images/stick.jpg">
Figure 3: Shows the proportions generated from the stick breaking process for two different values of $\alpha$ 1 and 5. We can see that for larger $\alpha$ the greater the number of clusters.



Section 1.4 Connection to the Dirichlet Process.

Above we gave three different ways to simulate a Dirichlet Process and now we can provide a formal definition. Given a base distribution $F_0$ and a dispersion parameter $\alpha$ a sample from a Dirichlet Process $DP(F_0, \alpha)$ is a distribution $F ~ DP(F_0, \alpha)$. The sample F can be thought of as one of the desnity functions generated from a single simulation of the Polya Urn Process shown above (or any of the other equivalent methods). Next we will show how to use Dirichlet Processes to perform clustering without having to specify the number of clusters a priori. This is a significant advantage in applications where one does not nescessarily know the number of clusters ahead of time.

Section 2. Dirichlet Process Mixture Model with Gibbs Sampling

Now we will discuss how to use Dirichlet Process to create an infinite mixture model for clustering. Specifically we will introduce an alternative to finite Gaussian Mixture Models which are typically implemented using the EM algorithm. We will consdier the case where our data is generated by some unknown number of gaussian distributions. Our goal is to cluster the data so that points that were generated from the same gaussian are clustered together. Note we will suppose that we do now know the number of cluster a priori but instead will learn this through the model. Figure 4 shows a greaphical model for the Dirichlet Process Mixture of Gaussian model.
<img src="images/dpmm_graphical_model.png">
Figure 4. Graphical Model for the Dirichlet Process Mixture Model. 


From this figure we can see that we assume each point that we observe $x_i$ has a latent variable $z_i$ that specifies which class generated that point and with each class there is an assocaited vector $\theta_c$ that specifies the parameters of that classes distribution. In our case we will assume that this is mean and covariance matrix of the normal distribution that generated the point $x_i$. Note that we also have two hyperparameters a dispersion parameter $\alpha$ where are larger value will lead to more clusters learned by the model and a base distribution $G_0$ which is chosen to be conjugate to the distributions used to generate each point (normal). We will use the Normal-inverse-Wishart for the base distribution.

In order to estimate the assignment for each datapoint $x_i$ we will use Gibbs Sampling. In general Gibbs Sampling follows this basic procedure:

* For each datapoint $x_i$ initialize a cluster assignment $z_i$
* Repeat the following steps until convergance
    * Randomly select a datapoint $x_i$ and keep all other cluster assignment $z_{j\neq i}$ fixed
    * Estimate a new cluster assignment $z_i$ by computing $Prob(z_i | z_{-i}, x_{1:n})$ for all possibilites and randomly select a new cluster assignment based on these probabilites
    
For the specific case of the Dirichlet Process Mixture of Gaussian Model the gibbs sampling update equations is as follows:

$Prob(z_i = k | z_{-i}, x_{1:n}, \alpha) = Prob(z_i = k | z_{-i}, \alpha)*Prob(x_i| z_{-i}, z_{i}=k, x_{-i})$

where

$Prob(z_i = k | z_{-i}, \alpha) = \frac{c_{k, -i}}{\alpha + n - 1}$ if k is an existing cluster and $c_{k, -i}$ is the number of points assignmed to cluster k with the exception of point i

and 

$Prob(z_i = k | z_{-i}, \alpha) = \frac{\alpha}{\alpha + n - 1}$ if k is a new cluster

and

$Prob(x_i| z_{-i}, z_{i}=k, x_{-i}) \propto N(\mu_{k, -i}, \Sigma_{k, -i})$


We implemented this procedure and tested for two different cases one where we the data was generated from two clusters and the other where the data was generated from four clusters. The original data the cluster assignments estimted from Gibbs sampling are showin in Figure 5. Note that Dirichlet Process does a very good job at determing the clusters of each datapoint without being told how many clusters there are a priori.


<img src="images/clusters.jpg">
Figure 5. Results from our implementation of the above model. The first row shows the case where our data was generated from 2 different gaussians the second row shows the case where our data was genreated from 4 different gaussians. The right column shows the raw data while the left column shows the results from the Infinite mixture model. Note how in both cases the model did an excellent job at determining the correct number of cluster and assigning points to the correct cluster.

Section 3. Hiearchial Dirichlet Process

Write about HDP

Section 4. Understanding a Corpus with Topic Models

The result of a topic model is essentially a probaility distribution over for each token of each document. This can be encoded as a matrix for each document in the corpus where the rows are over the words of the document and the columns are over the topics and the entry at position ij is the probability that word i was generated from topic j. From these matrices you can then infer the topics that make up a document and the words that make up a topic. However actually obtaining useful information from these distributions is very difficult to do. For the remainder of the paper we will introduce a variety of methods of understanding the results of topic models as well as combining the results with metadata (anything but the text of the documents) to draw useful information from a corpus of documents. Specifically we will focus on this task for a dataset of Hillary Clinton's emails. Some of the metadata we will be using is the senders and receivers of the emails, the times the emails were sent and several others.

It is not reasonable to expect that one will be able to learn much about a corpus from the raw numerical outputs of a trained topic model. In order to understand use topic models to better understand the structural themes (topics) of a corpus visualization systems need to be developed. Recently three such systems have been released as open source technology. The first being released in 2012 by David Blei's lab. The second, called Termite, released in 2012 by Jeffery Heer's lab and the third in 2014, called LDAviz, by Carson Sievert's lab. We will briefly discuss all three and then show the results of LDAviz (Carson Sievert's) for Hillary Clinton's email dataset. 

David Blei's lab released software that enabled users to navigate a topic model and use it to explore a corpus. The software begins by presenting the users with a list of topics where a topic is named after its top three words(the most likely words to generated from that topic). A user is then allowed to select a topic where they are navigated to a new page that shows document where that topic occurs with high probability as well as related topics. If a user selects on a document they are then navigated to a page contained the text from that document as well as pie graph showing the topic distriubtion of that specific document. This system does a very good job at letting a user explore the corpus at a high level. However significany improvements have been made in subsequent work.

One of the major improvements was made by the software Termite relased by Jeffery Heer's lab. The improvement made by Termite is how one orders the words of a topic. In David Blei's approach the words of a topic were ordered by probability but this can overweight words that occur frequently in the corpus. Instead Termite suggest to order words by their saliency. The saliency of word w is definted as follows:

$saliency(w) = Prob(w) * distinctiveness(w)$

where

$distinctiveness(w) = \sum_{T}Prob(T|w)log(\frac{Prob(T|w)}{Prob(T)})$

The idea behind saliency is that it gives greater weight to words that chatacterize a specific topic as opposed to belonging to multiple topics. 

The system developed by Carson Sievert LDAviz defines a similiar term to the saliency called the relevance. The relevance of a word w to a topic k with weight parameter $\lambda$ between 0 and 1 is defined as follows:

$relevancey(w, k | \lambda) = \lambda*log(Prob(w|k)) + (1 - \lambda)*log(\frac{Prob(w|k)}{Prob(w)})$

Note that setting $\lambda = 1$ reduces to ranking words of a topic according to their probability of being generated by that topic. Carson Sievert conducted a user study with 29 subjects (research scientist at AT&T Labs) and the user study suggested that $\lambda=0.6$ provided the best ordering of words in a topic. For this study the 20 New Group Dataset was used.

We used LDAviz to produce a high level overview of the Hillary Clinton email dataset. A snapshot of the visualization is shown in figure 6. To see the fully interactive version we produced open the file interactices.html in a modern web browser (we tested in Chrome). We will walk through the various components of the visualization but we hihgly recommend that one should explore the interactive version. The visualization opens by showing a bubble for each topic of the model. The size of the topic corresponds to its prevelance in the corpus and distance between topics encodes how similiar one topic is to another. The distance metric used between topics is Jensen-Shannon Divergence which is then projected onto a 2D plane using PCA. You can then select a specific topic and be shown the top ten words of that topic as determined by relevancey. The interactive version allows one to adjust the relevancy parameter and see how that affects the orderings of the words in the topic. One can then a select a specific word and the bubbles of other topics are highlighted if that words also appears frequently in them as well. This visuzalization provides an excellent way to understand the themes of a corpus. We recommend it as one of the first visualization to create when working with topic models and we also suggest that it should be consulted in tandem with the other visualization we created for the Hillary Clinton email dataset. 

<img src="images/topic_overview.jpg">
Figure 6: Shows the topic results of LDA (left) and HDP (right) trained on the Hilary Clinton email dataset. LDA was trained with 20 topics while HDP was given a max of no more than 50. The visualization is interactives (see interactives.html) and allows one to select a topic and view the 10 most probable workds. It also lets vary the relevancy parameter. The distance between topics is computed using Jensen-Shannon Divergence and is projected onto a 2D space using PCA.

Section 4.1 Bipartite Graph

Email sender and receiver is one type of metadata that the email data set provides. We have created a visualization for exploring the relationship between topics and indiviudal people. For each email we curate a list of the sender and all the receivers with the exception of Hillary Clinton. For each person we then create a list of all the emails that they sent or received and infer the topic distributions for all emails related to a specific person from a previously trained topic model. With these topic disbtributions we can compute the average probability that a specific topic was assigned to an a specific persons email to or from Hillary Clinton. 

We can visualize this information using a bipartie graph where one set of nodes is the people and the other set of nodes is the topic. There will be an edge from each person to every topic and the width of the edge is proportional to the average probability assigned to emails to or from that individual. Figure 7 shows an example of this visualization built using D3.js and with a LDA topic model using 20 topics. The figure shows two images one where all the people and all the topics are in the image and the other where we exclude all but one person and show that one persons relationship to all the topics. The image that includes all people and all topics is far to dense to gain useful information (unless you had giant monitor) while the other image showing all the topics and just one person is useful to explore the topics that the person is assocaited with. Note that this visualization is indtended to be used with the overview visualization of topics and words. One could also image building a similiar graph using groups of people. It could also be possible to find groups of people by clustering people on their probability of being associated with a specific topic, however this is not something we experimented with.

Note that the images of the bipartite graphs are just screenshots of the visualization we produced. The actual visualization we produced are interactive and directions for viewing the interactive versions can be found in the README.txt of this project.

<img src="images/bipartite.jpg">
Figure 7: Shows a bipartite graph where the green nodes are topics found by the model and the blue nodes are people who sent or received an email from Hillary Clinton. The width of an edge between people and topics is the average probability that the emails sent by that specific person contains that topic. The left graph shows all topics and all people and is far to dense to be useful. The right graph shows this for all topics and one specific person and thus can be used to explore what topics are prevelant in that persons emails with Hillary Clinton. 